In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_IGVF/3.Genome_wide_prediction/SCENT/SCENT_100.240614/pairs.E2G.res.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240610/sc-E2G/test/results/K562_IGVF_filter/K562/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_IGVF/1.prepare_data/2.Filter_out_low_quality_cells.240614/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_IGVF/3.Genome_wide_prediction/SCENT/SCENT_500.240615/"

In [3]:
n.cores = 48

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [5]:
matrix.atac = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [6]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Prepare SCENT meta data

In [7]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = "K562")
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
e10l1_AAACCAGGTCACAGGG-1,e10l1_AAACCAGGTCACAGGG-1,13033,3567,K562,9.475240,0.273689864
e10l1_AAACCGCCAAGGAATA-1,e10l1_AAACCGCCAAGGAATA-1,16605,2496,K562,9.717459,0.150316170
e10l1_AAACCGCCAGCCGGTA-1,e10l1_AAACCGCCAGCCGGTA-1,8953,1657,K562,9.099744,0.185077628
e10l1_AAACCGGTCAAGGAAG-1,e10l1_AAACCGGTCAAGGAAG-1,6886,246,K562,8.837246,0.035724659
e10l1_AAACGAACAACGTGGT-1,e10l1_AAACGAACAACGTGGT-1,22478,4700,K562,10.020292,0.209093336
e10l1_AAACGGGCAAGGGAGC-1,e10l1_AAACGGGCAAGGGAGC-1,10625,2370,K562,9.270965,0.223058824
e10l1_AAACTAGTCCTGTGAC-1,e10l1_AAACTAGTCCTGTGAC-1,7365,130,K562,8.904494,0.017651052
e10l1_AAACTAGTCGAAGCAT-1,e10l1_AAACTAGTCGAAGCAT-1,10650,2424,K562,9.273315,0.227605634
e10l1_AAACTAGTCGCTTGAC-1,e10l1_AAACTAGTCGCTTGAC-1,10031,247,K562,9.213436,0.024623667


RUN SCENT prediction

In [8]:
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        # nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        # nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        # if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            # bs = boot::boot(df2, assoc, R = 100, formula = formula, 
            #     stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            # p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
                bs = boot::boot(df2, assoc, R = 500, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
            #     bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        # }
    }
    object@SCENT.result <- res
    return(object)
}

In [9]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [10]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G)))
chr.run = chr.run[!chr.run %in% chr.done]

In [11]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr = pairs.E2G[seqnames(pairs.E2G) == chr.tmp]
  pairs.E2G.chr.res = pairs.E2G.chr[!is.na(pairs.E2G.chr$boot_basic_p) & pairs.E2G.chr$boot_basic_p < 0.1]
  pairs.E2G.chr.keep = pairs.E2G.chr[!pairs.E2G.chr %in% pairs.E2G.chr.res]
  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(n.cores))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna[rownames(matrix.rna) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(n.cores)
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  pairs.E2G.chr.merge = c(pairs.E2G.chr.res,
                          pairs.E2G.chr.keep)
  saveRDS(pairs.E2G.chr.merge,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.merge)
}

[1] "chr1"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr10"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr11"


...

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [12]:
chr.merge = as.character(unique(seqnames(pairs.E2G)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [13]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 118698 ranges and 11 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
  chr1.chr1     chr1 100037744-100039006      * |       FRRS1
  chr1.chr1     chr1 100037744-100039006      * |      LRRC39
  chr1.chr1     chr1 100132366-100133463      * |        DPH5
  chr1.chr1     chr1 100132366-100133463      * |       FRRS1
  chr1.chr1     chr1 100132366-100133463      * |      LRRC39
        ...      ...                 ...    ... .         ...
  chrX.chrX     chrX       989766-990603      * |       ASMTL
  chrX.chrX     chrX       989766-990603      * |      GTPBP6
  chrX.chrX     chrX       989766-990603      * |      PLCXD1
  chrX.chrX     chrX       989766-990603      * |     PPP2R3B
  chrX.chrX     chrX       989766-990603      * |     SLC25A6
                          PeakName               PairName       beta        se
                       <character>            <character

In [14]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100037744,100039006,FRRS1,K562,-0.07370103,0.02737191,-2.692579,7.090168e-03,0.044,1.3565473,-1,-1.3565473
chr1,100037744,100039006,LRRC39,K562,-0.50665207,0.14964712,-3.385645,7.101112e-04,0.004,2.3979400,-1,-2.3979400
chr1,100132366,100133463,DPH5,K562,0.14865865,0.05706449,2.605099,9.184773e-03,0.024,1.6197888,1,1.6197888
chr1,100132366,100133463,FRRS1,K562,-0.07010375,0.02618574,-2.677173,7.424639e-03,0.044,1.3565473,-1,-1.3565473
chr1,100132366,100133463,LRRC39,K562,-0.35594241,0.13506063,-2.635427,8.403156e-03,0.024,1.6197888,-1,-1.6197888
chr1,100132366,100133463,RTCA,K562,-0.09613838,0.04543866,-2.115784,3.436320e-02,0.064,1.1938200,-1,-1.1938200
chr1,100132366,100133463,SASS6,K562,-0.07497081,0.03051625,-2.456751,1.401999e-02,0.048,1.3187588,-1,-1.3187588
chr1,100132366,100133463,TRMT13,K562,0.09582672,0.04771733,2.008216,4.462031e-02,0.052,1.2839967,1,1.2839967
chr1,1001704,1002126,ATAD3A,K562,0.26863161,0.09790767,2.743724,6.074661e-03,0.016,1.7958800,1,1.7958800


In [15]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
[1] Matrix_1.6-5         GenomicRanges_1.54.1 GenomeInfoDb_1.38.1 
[4] IRanges_2.36.0       S4Vectors_0.40.2     BiocGeneri